In [35]:
import pandas as pd
import numpy as np

from src.utils.paths import DATA_DIR


In [36]:
df = pd.read_csv(DATA_DIR / "2025_june_raw.csv", encoding='cp1251')
df.head(2)


,Submitted at,Ваша основна зайнятість в ІТ зараз...,"ЗАРПЛАТА / СУМАРНИЙ ДОХІД в IT у $$$ за місяць, лише ставка \nЧИСТИМИ - після сплати податків",Всі бонуси (на місяць),Тайтл,Категорія,Почніть вводити і оберіть вашу ОСНОВНУ посаду зі списку,"У якій сфері проєкт, в якому ви зараз працюєте?",Вкажіть вашу основну спеціалізацію,Чи використовуєте ви у своїй роботі мови програмування (одну чи декілька)?,Основна мова програмування,"Основний напрям роботи компанії, в якій працюєте",Кількість спеціалістів у вашій компанії в Україні,Загальний стаж роботи за нинішньою ІТ-спеціальністю,Знання англійської мови,Де ви зараз живете?,За яким напрямом ви навчалися / навчаєтеся у виші?,Ваш вік
0,2025-05-26 06:52:09,Втратив(-ла) роботу / звільнився(-лася) і шука...,2000.0,0.0,Middle,Marketing,Marketing Manager,"Adtech / Advertising / Marketing, Blockchain, ...",NaN,"Ні, не використовую",NaN,Продуктова,до 50,4.0,Upper-Intermediate,В Україні,Інші гуманітарні та суспільні науки,21.0
1,2025-05-26 06:52:15,Працюю в IT-компанії / ІТ-департаменті,3000.0,0.0,Senior,SE,"Software Engineer / Developer (frontend, backe...",Retail / marketplace,Full Stack розробка,NaN,1С,Продуктова,до 10 спеціалістів,8.0,Pre-Intermediate,В Україні,"Точні науки (математика, інформатика)",33.0


In [37]:
# формую новий дф із ключовими фічами
selected_features = {
    'ЗАРПЛАТА / СУМАРНИЙ ДОХІД в IT у $$$ за місяць, лише ставка \nЧИСТИМИ - після сплати податків': 'salary_usd',
    'Тайтл': 'position_level',
    'Категорія': 'category',
    'Почніть вводити і оберіть вашу ОСНОВНУ посаду зі списку': 'position',
    'Знання англійської мови': 'english_level',
    'Загальний стаж роботи за нинішньою ІТ-спеціальністю': 'it_experience_years',
}

df_prepared = df[list(selected_features.keys())].copy()
df_prepared.rename(columns=selected_features, inplace=True)
df_prepared = df_prepared.dropna(subset=['salary_usd'])
df_prepared.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11685 entries, 0 to 11685
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   salary_usd           11685 non-null  float64
 1   position_level       11685 non-null  object 
 2   category             11685 non-null  object 
 3   position             11685 non-null  object 
 4   english_level        11685 non-null  object 
 5   it_experience_years  11685 non-null  float64
dtypes: float64(2), object(4)
memory usage: 639.0+ KB


## Preprocessing feature "Position Level"

In [38]:
df_prepared['position_level'].value_counts()


position_level
Middle                                   3869
Senior                                   3421
Junior                                   1472
Lead / Team Lead                          948
Manager                                   565
Немає тайтлу                              436
Tech Lead                                 290
Head                                      249
CEO / C-level (Chief) / Director / VP     152
Intern/Trainee                            107
Architect                                  96
Staff                                      42
Principal                                  38
Name: count, dtype: int64

In [39]:
level_mapping = {
    'Intern/Trainee': 'Intern',           # Pre-entry, learning phase
    'Junior': 'Junior',                    # 0-2 years, supervised work
    'Middle': 'Middle',                    # 2-4 years, independent work
    'Senior': 'Senior',                    # 4-7 years, expert level
    'Lead / Team Lead': 'Lead',            # Technical leadership track
    'Tech Lead': 'Lead',                   # Technical leadership track
    'Staff': 'Staff+',                     # Staff/Principal/Architect (IC track)
    'Principal': 'Staff+',
    'Architect': 'Staff+',
    'Manager': 'Management',               # People management track
    'Head': 'Management',
    'CEO / C-level (Chief) / Director / VP': 'C-level',  # C-suite separately
    'Немає тайтлу': 'Not Specified'
}
df_prepared['position_level'] = df_prepared['position_level'].map(level_mapping)
df_prepared['position_level'].value_counts()


position_level
Middle           3869
Senior           3421
Junior           1472
Lead             1238
Management        814
Not Specified     436
Staff+            176
C-level           152
Intern            107
Name: count, dtype: int64

In [40]:
ind_drop = df_prepared[df_prepared['position_level'] == 'Not Specified'].index
df_prepared.drop(ind_drop, inplace=True)


In [41]:
df_prepared['position_level'].value_counts()


position_level
Middle        3869
Senior        3421
Junior        1472
Lead          1238
Management     814
Staff+         176
C-level        152
Intern         107
Name: count, dtype: int64

## Preprocessing feature "position"

In [42]:
df_prepared['position'].value_counts()


position
Software Engineer / Developer (frontend, backend, mobile, gamedev, embedded etc )    4842
Manual QA                                                                            1166
DevOps Engineer                                                                       398
Project Manager (PM)                                                                  374
Automation QA / AQA                                                                   370
                                                                                     ... 
HR Analyst / People Analyst                                                             1
Web / UX analyst                                                                        1
Team Lead                                                                               1
Scriptwriter                                                                            1
Incident Manager                                                                        1
N

In [43]:
# Position to Category Mapping for Salary Prediction
# Optimized for Ukrainian IT market - Maximum 10 categories
# Focuses on high-level career tracks with minimal noise

position_category_map = {
    # ENGINEERING (Software Development, DevOps, Infrastructure)
    'Software Engineer / Developer (frontend, backend, mobile, gamedev, embedded etc )': 'Software Engineering',
    'Tech Lead / Engineering Lead': 'Software Engineering',
    'Engineering Manager': 'Software Engineering',
    'Engineering Lead': 'Software Engineering',
    'Head of Engineering': 'Software Engineering',
    'Head of Frontend': 'Software Engineering',
    'Head of Mobile': 'Software Engineering',
    'Head of Development': 'Software Engineering',
    'VP of Engineering': 'Software Engineering',
    'Director of Engineering': 'Software Engineering',
    'Team lead': 'Software Engineering',
    'Team Lead': 'Software Engineering',
    'DevOps Engineer': 'Software Engineering',
    'SRE (Site Reliability Engineer)': 'Software Engineering',
    'Infrastructure Engineer': 'Software Engineering',
    'Cloud Engineer': 'Software Engineering',
    'System engineer': 'Software Engineering',
    'DevOps Architect': 'Software Engineering',
    'Cloud Architect': 'Software Engineering',
    'Head of DevOps': 'Software Engineering',
    'Head of Cloud': 'Software Engineering',
    'Head of SRE': 'Software Engineering',
    'Head of DevOps & IT': 'Software Engineering',
    'DevOps Tech Lead': 'Software Engineering',
    'Software Architect': 'Software Engineering',
    'Solution Architect': 'Software Engineering',
    'System Architect': 'Software Engineering',
    'Application Architect': 'Software Engineering',
    'Security Architect': 'Software Engineering',
    'Integration Architect': 'Software Engineering',
    'Data Architect': 'Software Engineering',
    'Business Architect': 'Software Engineering',
    'Integrations Engineer / Integration Manager': 'Software Engineering',
    'Database Developer': 'Software Engineering',
    'DBRE (Database Reliability Engineer)': 'Software Engineering',
    'Application Manager': 'Software Engineering',
    'Configuration Engineer': 'Software Engineering',
    'Hardware Engineer': 'Software Engineering',
    'Electronics Engineer': 'Software Engineering',
    'Mechanical Design & Creative & Creative & Creative Engineer': 'Software Engineering',
    'Analog engineer (IC design)': 'Software Engineering',
    'TechOps Engineer': 'Software Engineering',

    # DATA SCIENCE & ML
    'Data Analyst': 'Data & Machine Learning',
    'Data Scientist': 'Data & Machine Learning',
    'Data Engineer / Big Data Engineer': 'Data & Machine Learning',
    'ML Engineer (Machine Learning)': 'Data & Machine Learning',
    'AI Engineer': 'Data & Machine Learning',
    'MLOps Engineer': 'Data & Machine Learning',
    'Computer Vision Engineer': 'Data & Machine Learning',
    'Computer Vision Engineer (CV)': 'Data & Machine Learning',
    'NLP Engineer': 'Data & Machine Learning',
    'Deep Learning Engineer': 'Data & Machine Learning',
    'Research Engineer': 'Data & Machine Learning',
    'Research Scientist': 'Data & Machine Learning',
    'BI Engineer': 'Data & Machine Learning',
    'BI Analyst': 'Data & Machine Learning',
    'Analytics Engineer': 'Data & Machine Learning',
    'Head of Analytics': 'Data & Machine Learning',
    'Analytics Team Lead': 'Data & Machine Learning',
    'Head of Data Science': 'Data & Machine Learning',
    'Head of AI': 'Data & Machine Learning',
    'AI Director': 'Data & Machine Learning',
    'CAO (Chief Analytics Officer)': 'Data & Machine Learning',
    'Product Analyst': 'Data & Machine Learning',
    'Marketing Data Analyst / Marketing Analyst': 'Data & Machine Learning',
    'Web Analyst': 'Data & Machine Learning',
    'Performance Analyst': 'Data & Machine Learning',
    'Research Analyst': 'Data & Machine Learning',
    'Operations Analyst': 'Data & Machine Learning',
    'Financial Analyst': 'Data & Machine Learning',
    'HR Analyst / People Analyst': 'Data & Machine Learning',
    'Strategy Analyst': 'Data & Machine Learning',
    'Game Analyst': 'Data & Machine Learning',
    'Anti-Fraud Analyst': 'Data & Machine Learning',
    'Web / UX analyst': 'Data & Machine Learning',

    # QA & TESTING
    'Manual QA': 'QA & Testing',
    'Automation QA / AQA': 'QA & Testing',
    'General QA': 'QA & Testing',
    'QA': 'QA & Testing',
    'SDET (Software Development Engineer in Test)': 'QA & Testing',
    'Software Quality Assurance Analyst': 'QA & Testing',
    'Data QA': 'QA & Testing',
    'Embedded QA': 'QA & Testing',
    'Performance QA': 'QA & Testing',
    'QA Lead': 'QA & Testing',
    'QA Manager / QC Manager': 'QA & Testing',
    'Head of QA': 'QA & Testing',
    'Quality Architect / Test Architect': 'QA & Testing',
    'QA TeamLead': 'QA & Testing',
    'Reverse Engineer': 'QA & Testing',

    # PRODUCT & PROJECT MANAGEMENT
    'Product Manager': 'Product & Project Management',
    'Product Owner (у продакт менеджменті)': 'Product & Project Management',
    'Product Owner (роль в скрамі)': 'Product & Project Management',
    'Technical Product Manager': 'Product & Project Management',
    'Head of Product': 'Product & Project Management',
    'CPO (Chief Product Officer)': 'Product & Project Management',
    'VP of Product': 'Product & Project Management',
    'Project Manager (PM)': 'Product & Project Management',
    'Project Coordinator': 'Product & Project Management',
    'Technical Project Manager': 'Product & Project Management',
    'Marketing Project Manager / Marketing Operations Manager': 'Product & Project Management',
    'Delivery Manager': 'Product & Project Management',
    'Delivery Director': 'Product & Project Management',
    'Head of Delivery': 'Product & Project Management',
    'Program Manager': 'Product & Project Management',
    'Program Director': 'Product & Project Management',
    'Release Manager': 'Product & Project Management',
    'Scrum Master': 'Product & Project Management',
    'Agile Coach': 'Product & Project Management',
    'Head of PMO': 'Product & Project Management',
    'Resource Manager': 'Product & Project Management',
    'Game Producer': 'Product & Project Management',
    'Producer / Creative Producer': 'Product & Project Management',
    'R&D Manager / RnD Manager': 'Product & Project Management',

    # DESIGN & CREATIVE
    'UI/UX Designer': 'Design & Creative',
    'Product Designer': 'Design & Creative',
    'UI/UX Artist': 'Design & Creative',
    'UI/UX Researcher': 'Design & Creative',
    'Web Designer': 'Design & Creative',
    'Graphic Designer': 'Design & Creative',
    'Graphic/Web designer': 'Design & Creative',
    'Ui/Ux and graphic designer': 'Design & Creative',
    'Brand Designer / Marketing Designer': 'Design & Creative',
    'Designer Generalist': 'Design & Creative',
    'Motion Designer': 'Design & Creative',
    'Head of UX/UI': 'Design & Creative',
    'Head of Design': 'Design & Creative',
    'Head of Product Design': 'Design & Creative',
    'Design team lead': 'Design & Creative',
    'Lead Product Designer': 'Design & Creative',
    'Creative Director': 'Design & Creative',
    '2D Artist': 'Design & Creative',
    '3D Artist': 'Design & Creative',
    'Generalist Artist': 'Design & Creative',
    'Technical Artist': 'Design & Creative',
    'Environment Artist': 'Design & Creative',
    'Concept Artist': 'Design & Creative',
    'VFX Artist': 'Design & Creative',
    'Animator 2D': 'Design & Creative',
    'Animator 3D': 'Design & Creative',
    'Technical Animator': 'Design & Creative',
    'Art Director': 'Design & Creative',
    'Game Designer': 'Design & Creative',
    'Technical Game Designer': 'Design & Creative',
    'UI/UX Game Designer': 'Design & Creative',
    'Level Designer': 'Design & Creative',
    'Narrative Designer': 'Design & Creative',
    'Game Economy Designer': 'Design & Creative',
    'Game Mathematician': 'Design & Creative',
    'Sound Designer': 'Design & Creative',
    'Lightibg Artist': 'Design & Creative',
    'UI/UX Writer': 'Design & Creative',

    # MARKETING & SALES
    'Marketing Manager': 'Marketing & Sales',
    'Head of Marketing': 'Marketing & Sales',
    'Product Marketing Manager': 'Marketing & Sales',
    'Content Manager / Content Marketing Manager': 'Marketing & Sales',
    'Content Writer': 'Marketing & Sales',
    'Head Of Content': 'Marketing & Sales',
    'Copywriter': 'Marketing & Sales',
    'SEO Specialist': 'Marketing & Sales',
    'Digital Marketing Manager': 'Marketing & Sales',
    'Email Marketing Specialist / E-mail Marketing specialist': 'Marketing & Sales',
    'Influencer marketing manager': 'Marketing & Sales',
    'SMM (Social Media Manager)': 'Marketing & Sales',
    'PPC Specialist': 'Marketing & Sales',
    'ASO Specialist / ASO Manager': 'Marketing & Sales',
    'Media Buyer': 'Marketing & Sales',
    'Marketing Assistant': 'Marketing & Sales',
    'Head of Digital Marketing': 'Marketing & Sales',
    'Growth Marketing Manager / Growth Manager': 'Marketing & Sales',
    'Head of PPC': 'Marketing & Sales',
    'Head of SEO': 'Marketing & Sales',
    'Head of SMM': 'Marketing & Sales',
    'Creative Marketing Specialist': 'Marketing & Sales',
    'Marketing Director': 'Marketing & Sales',
    'CMO (Chief Marketing Officer)': 'Marketing & Sales',
    'brand management': 'Marketing & Sales',
    'Brand Manager': 'Marketing & Sales',
    'Brand manager': 'Marketing & Sales',
    'Employer Brand Manager': 'Marketing & Sales',
    'User Acquisition Manager': 'Marketing & Sales',
    'Head of User Acquisition': 'Marketing & Sales',
    'CRM Manager': 'Marketing & Sales',
    'Head of CRM': 'Marketing & Sales',
    'Retention Manager': 'Marketing & Sales',
    'Head of Retention': 'Marketing & Sales',
    'Linkbuilder': 'Marketing & Sales',
    'SERM спеціаліст': 'Marketing & Sales',
    'Outreach Specialist': 'Marketing & Sales',
    'Scraping specialist': 'Marketing & Sales',
    'PR Manager': 'Marketing & Sales',
    'Head of PR': 'Marketing & Sales',
    'Communications Specialist': 'Marketing & Sales',
    'Head of Communications': 'Marketing & Sales',
    'Community Manager': 'Marketing & Sales',
    'Content Producer': 'Marketing & Sales',
    'Scriptwriter': 'Marketing & Sales',
    'Sales manager / Sales Executive / Sales Operations': 'Marketing & Sales',
    'Head of Sales': 'Marketing & Sales',
    'Account Manager / Account Executive': 'Marketing & Sales',
    'Account Executive (Sales)': 'Marketing & Sales',
    'Sales Development Representative': 'Marketing & Sales',
    'Lead Generation Specialist': 'Marketing & Sales',
    'Sales Director': 'Marketing & Sales',
    'Business Development Manager': 'Marketing & Sales',
    'Head of Business Development': 'Marketing & Sales',
    'CBDO (Chief Business Development Officer)': 'Marketing & Sales',
    'CCO (Chief Commercial Officer)': 'Marketing & Sales',
    'Presales Manager': 'Marketing & Sales',
    'Client Partner / Engagement manager': 'Marketing & Sales',
    'Customer Success Manager': 'Marketing & Sales',
    'Head of Customer Success': 'Marketing & Sales',
    'Head of Client Service': 'Marketing & Sales',
    'Head of Client Relations': 'Marketing & Sales',
    'Engagement lead': 'Marketing & Sales',
    'Partnership manager': 'Marketing & Sales',
    'Head of Partnerships': 'Marketing & Sales',
    'Head of Affiliates': 'Marketing & Sales',
    'Affiliate Manager': 'Marketing & Sales',
    'Event Manager': 'Marketing & Sales',

    # HR & PEOPLE OPERATIONS
    'Recruiter / Recruitment Specialist / Technical Recruiter': 'Human Resources & People Operations',
    'Talent Sourcer / Talent Researcher / Recruitment Researcher': 'Human Resources & People Operations',
    'Recruitment Assistant': 'Human Resources & People Operations',
    'Head of Recruitment': 'Human Resources & People Operations',
    'Talent Acquisition Manager': 'Human Resources & People Operations',
    'Head of Talent Acquisition': 'Human Resources & People Operations',
    'Talent Advisor': 'Human Resources & People Operations',
    'HR Manager / HR specialist': 'Human Resources & People Operations',
    'HR Director (HRD)': 'Human Resources & People Operations',
    'HR Generalist': 'Human Resources & People Operations',
    'HR People Partner': 'Human Resources & People Operations',
    'HR Business Partner': 'Human Resources & People Operations',
    'HR Coordinator': 'Human Resources & People Operations',
    'HR Administrator': 'Human Resources & People Operations',
    'HR Operations Specialist / People Operations Specialist': 'Human Resources & People Operations',
    'Head of HR': 'Human Resources & People Operations',
    'CHRO (Chief Human Resources Officer)': 'Human Resources & People Operations',
    'HR Lead': 'Human Resources & People Operations',
    'L&D Lead / Learning & Development Lead': 'Human Resources & People Operations',
    'Learning and Development / Training and Development': 'Human Resources & People Operations',
    'Head of Learning and Development Department': 'Human Resources & People Operations',
    'Senior Learning Experience Designer': 'Human Resources & People Operations',
    'Education Manager': 'Human Resources & People Operations',
    'Teacher / Lecturer': 'Human Resources & People Operations',
    'Employee Engagement Manager /Employee experience Manager': 'Human Resources & People Operations',
    'Compensation and Benefit Specialist': 'Human Resources & People Operations',
    'HR Marketing Specialist': 'Human Resources & People Operations',

    # SECURITY & COMPLIANCE
    'Head of Information Security': 'Security & Compliance',
    'Security Engineer': 'Security & Compliance',
    'Information security specialist': 'Security & Compliance',
    'Security Analyst / SOC Analyst': 'Security & Compliance',
    'SOC Engineer / SecOps Engineer': 'Security & Compliance',
    'SOC Automation Engineer': 'Security & Compliance',
    'Application security engineer': 'Security & Compliance',
    'Cybersecurity Engineer': 'Security & Compliance',
    'Network Security Engineer': 'Security & Compliance',
    'CSO (Chief Security Officer)': 'Security & Compliance',
    'Penetration Tester': 'Security & Compliance',
    'Penetration Tester / Pentester': 'Security & Compliance',
    'Compliance Manager': 'Security & Compliance',
    'Risk manager': 'Security & Compliance',
    'IT Auditor': 'Security & Compliance',
    'Internal Auditor': 'Security & Compliance',

    # IT OPERATIONS & SUPPORT
    'SysAdmin': 'IT Operations',
    'DBA (Database administrator)': 'IT Operations',
    'Network Engineer': 'IT Operations',
    'NOC Engineer': 'IT Operations',
    'System Analyst': 'IT Operations',
    'monitoring engineer': 'IT Operations',
    'Technical Support': 'IT Operations',
    'Head of Support': 'IT Operations',
    'Customer Support': 'IT Operations',
    'Head of Customer Support': 'IT Operations',
    'customer support general manager': 'IT Operations',
    'CSR Lead': 'IT Operations',
    'Helpdesk Engineer': 'IT Operations',
    'Service Desk Specialist': 'IT Operations',
    'IT Director': 'IT Operations',
    'Head of IT Sector': 'IT Operations',
    'Deputy IT Manager': 'IT Operations',
    'СIO (Chief information Officer)': 'IT Operations',
    'CTO (Chief Technology Officer) / Technical Director': 'IT Operations',
    'ERP/CRM/SAP Consultant': 'IT Operations',
    'Salesforce Consultant': 'IT Operations',
    'Localization Manager': 'IT Operations',
    'Technical writer': 'IT Operations',
    'Technical writer / Techwriter': 'IT Operations',
    'Incident Manager': 'IT Operations',
    'Payments Manager': 'IT Operations',
    'Менеджер з аудіоконтролю': 'IT Operations',

    # BUSINESS & OPERATIONS (Non-technical roles)
    'Business Analyst (BA)': 'Business/Commercial Operations',
    'Head of BA Office': 'Business/Commercial Operations',
    'Operations Manager': 'Business/Commercial Operations',
    'Operations Director': 'Business/Commercial Operations',
    'Head of Operations': 'Business/Commercial Operations',
    'COO (Chief Operating Officer)': 'Business/Commercial Operations',
    'Office Manager / Office Administrator / Office Coordinator': 'Business/Commercial Operations',
    'Executive Assistant': 'Business/Commercial Operations',
    'Administrative Assistant': 'Business/Commercial Operations',
    'Administrative Manager': 'Business/Commercial Operations',
    'Accountant': 'Business/Commercial Operations',
    'Finance Manager / Financial Manager / Financial controller': 'Business/Commercial Operations',
    'CFO (Chief Financial Officer)': 'Business/Commercial Operations',
    'Head of Finance': 'Business/Commercial Operations',
    'Treasury Manager': 'Business/Commercial Operations',
    'Billing Operations Specialist / Payment Operations / P2P Specialist (Procurement)': 'Business/Commercial Operations',
    'Lawyer': 'Business/Commercial Operations',
    'Legal Counsel': 'Business/Commercial Operations',
    'CLO (Chief Legal Officer)': 'Business/Commercial Operations',
    'Head of Legal': 'Business/Commercial Operations',
    'CEO / founder / cofounder': 'Business/Commercial Operations',
    'CEO': 'Business/Commercial Operations',
    'Country Manager': 'Business/Commercial Operations',
    'CAO (Chief Administrative Officer)': 'Business/Commercial Operations',
    'Head of Corporate Strategy': 'Business/Commercial Operations',
}


In [44]:
df_prepared['pos_to_cat'] = df_prepared['position'].map(position_category_map)
df_prepared['pos_to_cat'].value_counts()


pos_to_cat
Software Engineering                   5690
QA & Testing                           1977
Product & Project Management            820
Data & Machine Learning                 538
Marketing & Sales                       494
Design & Creative                       489
Human Resources & People Operations     486
Business/Commercial Operations          364
IT Operations                           323
Security & Compliance                    67
Name: count, dtype: int64

In [45]:
df_prepared.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11249 entries, 0 to 11685
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   salary_usd           11249 non-null  float64
 1   position_level       11249 non-null  object 
 2   category             11249 non-null  object 
 3   position             11249 non-null  object 
 4   english_level        11249 non-null  object 
 5   it_experience_years  11249 non-null  float64
 6   pos_to_cat           11248 non-null  object 
dtypes: float64(2), object(5)
memory usage: 703.1+ KB


In [46]:
# Виділяємо три групи
se_df = df_prepared[df_prepared['pos_to_cat'] == 'Software Engineering']
qa_df = df_prepared[df_prepared['pos_to_cat'] == 'QA & Testing']
other_df = df_prepared[~df_prepared['pos_to_cat'].isin(['Software Engineering', 'QA & Testing'])]

# Стратифікований sampling для SE
se_sampled = se_df.groupby(['position_level', 'salary_usd'], group_keys=False).apply(
    lambda x: x.sample(min(len(x), max(1, int(100 * len(x) / len(se_df)))),
                       random_state=42)
)

# Стратифікований sampling для QA
qa_sampled = qa_df.groupby(['position_level', 'salary_usd'], group_keys=False).apply(
    lambda x: x.sample(min(len(x), max(1, int(600 * len(x) / len(qa_df)))),
                       random_state=42)
)

# Об'єднуємо всі три групи
balanced_df = pd.concat([se_sampled, qa_sampled, other_df], ignore_index=True)

print("Було SE:", len(se_df), "Стало SE:", len(se_sampled))
print("Було QA:", len(qa_df), "Стало QA:", len(qa_sampled))
print("Other:", len(other_df))
print("\nРозподіл після балансування:")
print(balanced_df['pos_to_cat'].value_counts())
print(f"\nЗагальна кількість: {len(balanced_df)}")


/var/folders/gc/s2x5tzvj4dz9sgc9x_cvglz80000gn/T/ipykernel_1526/1707969274.py:7: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Було SE: 5690 Стало SE: 1059
Було QA: 1977 Стало QA: 832
Other: 3582

Розподіл після балансування:
pos_to_cat
Software Engineering                   1059
QA & Testing                            832
Product & Project Management            820
Data & Machine Learning                 538
Marketing & Sales                       494
Design & Creative                       489
Human Resources & People Operations     486
Business/Commercial Operations          364
IT Operations                           323
Security & Compliance                    67
Name: count, dtype: int64

Загальна кількість: 5473


/var/folders/gc/s2x5tzvj4dz9sgc9x_cvglz80000gn/T/ipykernel_1526/1707969274.py:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



## Preprocessing feature "English Level"

In [47]:
balanced_df['english_level'].value_counts()


english_level
Upper-Intermediate    2424
Intermediate          1314
Advanced              1087
Pre-Intermediate       500
Elementary             138
Не знаю взагалі         10
Name: count, dtype: int64

In [48]:
engl_ind = balanced_df[balanced_df['english_level'] == 'Не знаю взагалі'].index
balanced_df.drop(engl_ind, inplace=True)
balanced_df['english_level'].value_counts()


english_level
Upper-Intermediate    2424
Intermediate          1314
Advanced              1087
Pre-Intermediate       500
Elementary             138
Name: count, dtype: int64

In [49]:
balanced_df.head(3)


,salary_usd,position_level,category,position,english_level,it_experience_years,pos_to_cat
0,2000.0,C-level,SE,"Software Engineer / Developer (frontend, backe...",Intermediate,20.0,Software Engineering
1,4500.0,C-level,SE,"Software Engineer / Developer (frontend, backe...",Pre-Intermediate,18.0,Software Engineering
2,6000.0,C-level,Tech Leadership,VP of Engineering,Intermediate,20.0,Software Engineering


In [50]:
dataframe = balanced_df[['pos_to_cat', 'position_level', 'english_level', 'it_experience_years', 'salary_usd']].copy()


In [51]:
dataframe


,pos_to_cat,position_level,english_level,it_experience_years,salary_usd
0,Software Engineering,C-level,Intermediate,20.0,2000.0
1,Software Engineering,C-level,Pre-Intermediate,18.0,4500.0
2,Software Engineering,C-level,Intermediate,20.0,6000.0
3,Software Engineering,C-level,Upper-Intermediate,14.0,6800.0
4,Software Engineering,C-level,Advanced,18.0,8000.0
...,...,...,...,...,...
5468,Design & Creative,Middle,Upper-Intermediate,2.0,1700.0
5469,Data & Machine Learning,Senior,Advanced,6.0,5220.0
5470,Product & Project Management,Management,Advanced,6.0,3850.0
5471,IT Operations,Middle,Upper-Intermediate,1.5,2500.0


## Outliers cleaning (Experience/Salary & Position Level/Experience)

In [52]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.figure_factory as ff


In [53]:
fig = px.scatter(
    dataframe,
    x='it_experience_years',
    y='salary_usd',
    color='position_level',
    hover_data=['pos_to_cat','it_experience_years', 'salary_usd'],
    title='Experience vs position_level',
    width=900, height=500
).show()


Очищення даних, так як певна частина даних являється не інформативною для моделі або ж фейковою (наприклад: інтерн, з досвідом 20 років)

In [54]:
# формування діапазонів для перевіки на коректність даних
ukraine_experience_ranges = {
    'Intern': {
        'typical_range': (0, 1),
        'acceptable': (0, 2),
        'outlier_threshold': 3,        # 3+ років досвіду для інтерна — малоймовірно
        'critical_outlier': 4,
        'note': 'Стажери, студенти, випускники без комерційного досвіду'
    },

    'Junior': {
        'typical_range': (0.5, 2.5),
        'acceptable': (0, 3.5),
        'outlier_threshold': 4,        # 4+ років для джуна — підозріло
        'critical_outlier': 5,
        'note': 'Початковий рівень із базовим продакшн-досвідом'
    },

    'Middle': {
        'typical_range': (2, 5),
        'acceptable': (1.5, 7),
        'outlier_threshold': 8,        # 8+ років без росту — нетипово
        'critical_outlier': 10,
        'note': 'Фахівці середнього рівня, автономна робота над фічами'
    },

    'Senior': {
        'typical_range': (4, 9),
        'acceptable': (3, 12),
        'outlier_threshold': 2,        # менше 3 років досвіду — підозріло
        'critical_outlier': 1,
        'max_outlier': 20,             # більше 20 років — теж дивно
        'note': 'Досвідчені інженери, менторство, відповідальність за модулі/команди'
    },

    'Lead': {
        'typical_range': (6, 12),
        'acceptable': (5, 15),
        'outlier_threshold': 3,        # лід із 3 роками — фейк
        'critical_outlier': 2,
        'max_outlier': 20,
        'note': 'Технічні лідери, відповідальні за команду або архітектуру'
    },

    'Staff+': {
        'typical_range': (8, 15),
        'acceptable': (6, 18),
        'outlier_threshold': 5,
        'critical_outlier': 3,
        'max_outlier': 25,
        'note': 'Висококваліфіковані експерти, архітектори, технічні радники'
    },

    'Management': {
        'typical_range': (7, 15),
        'acceptable': (5, 18),
        'outlier_threshold': 4,
        'critical_outlier': 3,
        'max_outlier': 25,
        'note': 'Керівники команд, delivery/engineering managers'
    },

    'C-level': {
        'typical_range': (12, 25),
        'acceptable': (10, 30),
        'outlier_threshold': 7,        # C-level з 7 роками — підозріло
        'critical_outlier': 5,
        'max_outlier': 40,
        'note': 'ТОП-менеджмент: CTO, CIO, CDO, CEO у технічних компаніях'
    }
}


In [55]:
# перевірка роботи логіки
for index, row in dataframe.iloc[90:94].iterrows():
    title = row['position_level']
    experience = row['it_experience_years']

    print(title, experience)

    if title in ukraine_experience_ranges:
        rules = ukraine_experience_ranges[title]
        print(rules)

        criticar_outlier = rules['critical_outlier']
        outlier_threshold = rules['outlier_threshold']
        acceptable_threshold = rules['acceptable']
        typical_range = rules['typical_range']

        #  першочергово перевіряємо на наявність max_outlier (присутній не в усіх ключах)
        if 'max_outlier' in rules and experience > rules['max_outlier']:
            print(f'st.5 - bad')
            score = 1.0
        # перевірка на наявність у типовому діапазоні
        elif typical_range[0] <= experience <= typical_range[1]:
            print(f'St.1: {experience} is within typical range. (ok)')
            score = 0.0
        # ще більш-менш прийнятний діапазон
        elif acceptable_threshold[0] <= experience <= acceptable_threshold[1]:
            print(f'St.2: {experience} is within acceptable threshold. (ok)')
            score = 0.25
        elif experience < criticar_outlier:
            print(f'St.4: {experience} < {criticar_outlier}')
            score = 0.75
        elif experience < outlier_threshold:
            print(f'St.3: {experience} < {outlier_threshold}. (bad)')
            score = 0.50
        else:
            print(f'St.6: {experience} is outside all ranges')
            score = 1.0
        print(f'Total score: {score}\n')


Junior 0.1
{'typical_range': (0.5, 2.5), 'acceptable': (0, 3.5), 'outlier_threshold': 4, 'critical_outlier': 5, 'note': 'Початковий рівень із базовим продакшн-досвідом'}
St.2: 0.1 is within acceptable threshold. (ok)
Total score: 0.25

Junior 2.0
{'typical_range': (0.5, 2.5), 'acceptable': (0, 3.5), 'outlier_threshold': 4, 'critical_outlier': 5, 'note': 'Початковий рівень із базовим продакшн-досвідом'}
St.1: 2.0 is within typical range. (ok)
Total score: 0.0

Junior 5.0
{'typical_range': (0.5, 2.5), 'acceptable': (0, 3.5), 'outlier_threshold': 4, 'critical_outlier': 5, 'note': 'Початковий рівень із базовим продакшн-досвідом'}
St.6: 5.0 is outside all ranges
Total score: 1.0

Junior 1.5
{'typical_range': (0.5, 2.5), 'acceptable': (0, 3.5), 'outlier_threshold': 4, 'critical_outlier': 5, 'note': 'Початковий рівень із базовим продакшн-досвідом'}
St.1: 1.5 is within typical range. (ok)
Total score: 0.0



In [56]:
def detect_experience_outliers(df):

    # base
    df_copy = df.copy()
    df_copy['is_outlier'] = False
    df_copy['outlier_score'] = float(0.0)

    # even row
    for index, row in df_copy.iterrows():
        title = row['position_level'] # кожен рядок title
        experience = row['it_experience_years'] # кожен рядок experience

        score = 0.0

        if title in ukraine_experience_ranges:
            rules = ukraine_experience_ranges[title]

            # визначаємо діапазони для викидів
            critical_outlier = rules['critical_outlier']
            outlier_threshold = rules['outlier_threshold']
            acceptable_threshold = rules['acceptable'] # ok
            typical_range = rules['typical_range'] # ok

            if 'max_outlier' in rules and experience > rules['max_outlier']:
                score = 1.0
            elif typical_range[0] <= experience <= typical_range[1]:
                score = 0.0
            elif acceptable_threshold[0] <= experience <= acceptable_threshold[1]:
                score = 0.25
            elif experience < critical_outlier:
                score = 0.75
            elif experience < outlier_threshold:
                score = 0.50
            else:
                score = 1.0

        if score >= 0.5:
            df_copy.at[index, 'is_outlier'] = True
        df_copy.at[index, 'outlier_score'] = score

    return df_copy


In [57]:
df_exp_outliers = detect_experience_outliers(dataframe)
df_exp_outliers['is_outlier'].value_counts()


is_outlier
False    4454
True     1009
Name: count, dtype: int64

In [58]:
fig = px.scatter(
    df_exp_outliers,
    x='it_experience_years',
    y='salary_usd',
    color='is_outlier',
    hover_data=['position_level', 'pos_to_cat','it_experience_years', 'salary_usd'],
    title='Outliers cleaning in "IT_Experience"',
    width=900, height=500,
)

fig.show()


In [59]:
ukraine_salary_ranges = {
    'Intern': {
        'typical_range': (300, 600),         # стажери або часткова зайнятість
        'acceptable': (200, 800),
        'outlier_range': (150, 1000),        # нижче 150 або вище 1000 — підозріло
        'critical_range': (100, 1200),       # явно нереалістичні значення
        'note': 'Стажери, студенти, початкова практика'
    },

    'Junior': {
        'typical_range': (700, 1500),        # типовий джун
        'acceptable': (500, 1800),
        'outlier_range': (400, 2200),        # нижче 400 або вище 2200 — підозріло
        'critical_range': (300, 2500),       # поза межами цього — нереалістично
        'note': 'Джуни, які мають базовий досвід продакшн'
    },

    'Middle': {
        'typical_range': (1600, 3000),       # основна маса мідлів
        'acceptable': (1300, 3800),
        'outlier_range': (1000, 4500),       # нижче 1k або вище 4.5k — підозріло
        'critical_range': (800, 5500),       # явна помилка
        'note': 'Фахівці середнього рівня (Middle Engineer / Analyst)'
    },

    'Senior': {
        'typical_range': (3000, 5500),       # сеньйори
        'acceptable': (2500, 7000),
        'outlier_range': (2000, 8500),       # нижче 2k або вище 8.5k — червоний прапорець
        'critical_range': (1500, 10000),
        'note': 'Досвідчені розробники або технічні експерти'
    },

    'Lead': {
        'typical_range': (4500, 7500),       # тімліди, техліди
        'acceptable': (3500, 9000),
        'outlier_range': (2500, 10000),      # нижче 2.5k або вище 10k — підозріло
        'critical_range': (2000, 12000),
        'note': 'Технічні лідери, керівники команд'
    },

    'Staff+': {
        'typical_range': (6000, 9500),       # архітектори, експерти
        'acceptable': (5000, 12000),
        'outlier_range': (4000, 14000),      # нижче 4k або вище 14k — підозріло
        'critical_range': (3000, 16000),
        'note': 'Архітектори, Staff/Principal Engineers, експерти високого рівня'
    },

    'Management': {
        'typical_range': (5000, 9000),       # Engineering Manager, Head of Dept.
        'acceptable': (4000, 11000),
        'outlier_range': (3000, 13000),      # нижче 3k або вище 13k — дивно
        'critical_range': (2500, 15000),
        'note': 'Керівники команд, Delivery або Engineering Managers'
    },

    'C-level': {
        'typical_range': (9000, 18000),      # CTO, CIO, CDO, CEO у тех-компаніях
        'acceptable': (7000, 22000),
        'outlier_range': (5000, 25000),      # нижче 5k або вище 25k — підозріло
        'critical_range': (4000, 30000),
        'note': 'ТОП-менеджмент компаній або засновники із техбекграундом'
    }
}


In [60]:
# для перевірки
for index, row in df_exp_outliers[1000:1002].iterrows():
    title = row['position_level']
    salary = row['salary_usd']
    print(title, salary)

    score = float(0.0)

    if title in ukraine_salary_ranges:
        rules = ukraine_salary_ranges[title]
        print(rules)

        acceptable_threshold = rules['acceptable']
        typical_range = rules['typical_range']
        outlier_threshold = rules['outlier_range']
        critical_range = rules['critical_range']

        if typical_range[0] <= salary <= typical_range[1]:
            print(f'st.1: {typical_range[0]} <= {salary} <= {typical_range[1]}')
            score = 0.0
        elif acceptable_threshold[0] <= salary <= acceptable_threshold[1]:
            print(f'st.2: {acceptable_threshold[0]} <= {salary} <= {acceptable_threshold[1]}')
            score = 0.25
        elif outlier_threshold[0] <= salary <= outlier_threshold[1]:
            print(f'st.3: {outlier_threshold[0]} <= {salary} <= {outlier_threshold[1]}')
            score = 0.75
        elif critical_range[0] <= salary <= critical_range[1]:
            print(f'st.4: {critical_range[0]} <= {salary} <= {critical_range[1]}')
            score = 1.0
        else:
            score = 1.0

        print(f'Total score for {title}: {salary:.0f}$ = {score}\n')


Staff+ 4590.0
{'typical_range': (6000, 9500), 'acceptable': (5000, 12000), 'outlier_range': (4000, 14000), 'critical_range': (3000, 16000), 'note': 'Архітектори, Staff/Principal Engineers, експерти високого рівня'}
st.3: 4000 <= 4590.0 <= 14000
Total score for Staff+: 4590$ = 0.75

Staff+ 4650.0
{'typical_range': (6000, 9500), 'acceptable': (5000, 12000), 'outlier_range': (4000, 14000), 'critical_range': (3000, 16000), 'note': 'Архітектори, Staff/Principal Engineers, експерти високого рівня'}
st.3: 4000 <= 4650.0 <= 14000
Total score for Staff+: 4650$ = 0.75



In [61]:
def detect_salary_outliers(df):
    """Повертає score від 0.0 (норма) до 1.0 (критичний викид)."""

    df_copy = df.copy()
    df_copy['is_salary_outlier'] = False
    df_copy['outlier_salary_score'] = float(0.0)

    for index, row in df_copy.iterrows():
        title = row['position_level']
        salary = row['salary_usd']

        score = float(0.0)

        if title in ukraine_salary_ranges:
            rules = ukraine_salary_ranges[title]

            acceptable_threshold = rules['acceptable']
            typical_range = rules['typical_range']
            outlier_threshold = rules['outlier_range']
            critical_range = rules['critical_range']

            if typical_range[0] <= salary <= typical_range[1]:
                score = 0.0
            elif acceptable_threshold[0] <= salary <= acceptable_threshold[1]:
                score = 0.25
            elif outlier_threshold[0] <= salary <= outlier_threshold[1]:
                score = 0.75
            elif critical_range[0] <= salary <= critical_range[1]:
                score = 1.0
            else:
                score = 1.0

        if score >= 0.5:
            df_copy.at[index, 'is_salary_outlier'] = True
        df_copy.at[index, 'outlier_salary_score'] = score

    return df_copy


In [62]:
df_slr_outlier = detect_salary_outliers(df_exp_outliers)
df_slr_outlier['is_salary_outlier'].value_counts()


is_salary_outlier
False    3482
True     1981
Name: count, dtype: int64

In [63]:
#  комбінований стовпець
df_slr_outlier['outlier_status'] = df_slr_outlier.apply(
    lambda row: (
        'Both Outliers' if row['is_outlier'] and row['is_salary_outlier'] else
        'Experience Outlier' if row['is_outlier'] else
        'Salary Outlier' if row['is_salary_outlier'] else
        'Normal'
    ),
    axis=1
)


In [64]:
df_slr_outlier['outlier_status'].value_counts()


outlier_status
Normal                3044
Salary Outlier        1410
Both Outliers          571
Experience Outlier     438
Name: count, dtype: int64

In [65]:
fig = px.scatter(
    df_slr_outlier,
    x='it_experience_years',
    y='salary_usd',
    hover_data=['salary_usd', 'position_level'],
    color='outlier_status',
)
fig.add_shape(
    type='rect',
    xref='paper',
    yref='y',
    x0=0, x1=1,
    y0=0, y1=1500,
    fillcolor='LightSalmon',
    opacity=0.3,
    layer='below',
    line_width=0
)
fig.show()


In [66]:
data = df_slr_outlier[df_slr_outlier['outlier_status'] == 'Normal'].copy()
data = data[['pos_to_cat', 'position_level', 'english_level', 'it_experience_years', 'salary_usd']].copy()
data = data.rename(columns={
    'pos_to_cat': 'job_category',
    'position_level': 'seniority_level',
    'it_experience_years': 'experience_years'})
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 3044 entries, 4 to 5472
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   job_category      3044 non-null   object 
 1   seniority_level   3044 non-null   object 
 2   english_level     3044 non-null   object 
 3   experience_years  3044 non-null   float64
 4   salary_usd        3044 non-null   float64
dtypes: float64(2), object(3)
memory usage: 142.7+ KB


In [67]:
data


,job_category,seniority_level,english_level,experience_years,salary_usd
4,Software Engineering,C-level,Advanced,18.0,8000.0
9,Software Engineering,C-level,Advanced,14.0,14000.0
10,Software Engineering,C-level,Advanced,10.0,14664.0
14,Software Engineering,Intern,Intermediate,0.1,200.0
15,Software Engineering,Intern,Upper-Intermediate,0.1,210.0
...,...,...,...,...,...
5467,Product & Project Management,Middle,Upper-Intermediate,3.0,3000.0
5468,Design & Creative,Middle,Upper-Intermediate,2.0,1700.0
5469,Data & Machine Learning,Senior,Advanced,6.0,5220.0
5471,IT Operations,Middle,Upper-Intermediate,1.5,2500.0


In [68]:
data.to_csv(DATA_DIR / "data_v4.csv", index=False)
